In [1]:
import sys

sys.path.append('/home/azhuravl/work/shape-of-motion')

import flow3d.data.iphone_dataset as iphone_dataset

In [ ]:
import torch

train_dataset = iphone_dataset.iPhoneDataset(
    '/home/azhuravl/nobackup/shape_of_motion/paper-windmill',
    split='train',
    scene_norm_dict=torch.load('/home/azhuravl/nobackup/shape_of_motion/paper-windmill/flow3d_preprocessed/cache/scene_norm_dict.pth', weights_only=True),
    # skip_load_imgs=True,
)

test_dataset = iphone_dataset.iPhoneDataset(
    '/home/azhuravl/nobackup/shape_of_motion/paper-windmill',
    split='val',
    scene_norm_dict=torch.load('/home/azhuravl/nobackup/shape_of_motion/paper-windmill/flow3d_preprocessed/cache/scene_norm_dict.pth', weights_only=True),
    skip_load_imgs=True,
)
    

2025-09-12 20:40:50.544 | INFO     | flow3d.data.iphone_dataset:__init__:113 - self.time_ids.min()=tensor(0) self.time_ids.max()=tensor(276)
2025-09-12 20:40:50.546 | INFO     | flow3d.data.iphone_dataset:__init__:114 - self.num_frames=277


No monocular normal found


2025-09-12 20:42:00.391 | INFO     | flow3d.data.iphone_dataset:__init__:379 - len(self.query_tracks_2d)=277 self.query_tracks_2d[0].shape=torch.Size([1739, 4])
2025-09-12 20:42:00.425 | INFO     | flow3d.data.iphone_dataset:__init__:460 - self.imgs.shape=torch.Size([277, 960, 720, 3]) self.valid_masks.shape=torch.Size([277, 960, 720]) self.masks.shape=torch.Size([277, 960, 720])


In [11]:
# for each entry in dataset[0], print its name and shape, and value if shape doesn't exist
for name, value in train_dataset[0].items():
    if hasattr(value, 'shape'):
        print(f'{name}: {value.shape}')
    else:
        print(f'{name}: {value}')

frame_names: 0_00000
ts: torch.Size([])
w2cs: torch.Size([4, 4])
Ks: torch.Size([3, 3])
imgs: torch.Size([960, 720, 3])
valid_masks: torch.Size([960, 720])
masks: torch.Size([960, 720])
depths: torch.Size([960, 720])
query_tracks_2d: torch.Size([1739, 2])
target_ts: torch.Size([1])
target_w2cs: torch.Size([1, 4, 4])
target_Ks: torch.Size([1, 3, 3])
target_tracks_2d: torch.Size([1, 1739, 2])
target_visibles: torch.Size([1, 1739])
target_invisibles: torch.Size([1, 1739])
target_confidences: torch.Size([1, 1739])
target_track_depths: torch.Size([1, 1739])


In [5]:
def find_contiguous_sequences(train_dataset, test_dataset, camera_pairs=[(0, 1), (0, 2)], sequence_length=49):
    """
    Find contiguous sequences of frames for specified camera pairs
    
    Args:
        train_dataset: Dataset containing camera 0 frames
        test_dataset: Dataset containing camera 1 and 2 frames  
        camera_pairs: List of (source_cam, target_cam) tuples to check
        sequence_length: Required sequence length (default 49)
    
    Returns:
        dict: Results for each camera pair with frame names and dataset indices
    """
    
    # Get all available frames for each camera
    train_frames = train_dataset.frame_names  # Camera 0 frames
    test_frames = test_dataset.frame_names   # Camera 1 and 2 frames
    
    # Parse frame names to get camera_id and timestep
    def parse_frame_name(frame_name):
        parts = frame_name.split('_')
        camera_id = int(parts[0])
        timestep = int(parts[1])
        return camera_id, timestep
    
    # Group frames by camera and timestep
    camera_frames = {}
    
    # Process train dataset (camera 0)
    for idx, frame_name in enumerate(train_frames):
        cam_id, timestep = parse_frame_name(frame_name)
        if cam_id not in camera_frames:
            camera_frames[cam_id] = {}
        camera_frames[cam_id][timestep] = {
            'frame_name': frame_name,
            'dataset': 'train',
            'dataset_idx': idx
        }
    
    # Process test dataset (cameras 1 and 2)
    for idx, frame_name in enumerate(test_frames):
        cam_id, timestep = parse_frame_name(frame_name)
        if cam_id not in camera_frames:
            camera_frames[cam_id] = {}
        camera_frames[cam_id][timestep] = {
            'frame_name': frame_name,
            'dataset': 'test', 
            'dataset_idx': idx
        }
    
    print(f"Available cameras: {list(camera_frames.keys())}")
    for cam_id in camera_frames.keys():
        timesteps = sorted(camera_frames[cam_id].keys())
        print(f"Camera {cam_id}: {len(timesteps)} frames, timesteps {min(timesteps)}-{max(timesteps)}")
    
    # Find contiguous sequences for each camera pair
    results = {}
    
    for source_cam, target_cam in camera_pairs:
        print(f"\n=== Checking camera pair ({source_cam}, {target_cam}) ===")
        
        if source_cam not in camera_frames:
            print(f"❌ Camera {source_cam} not found")
            continue
            
        if target_cam not in camera_frames:
            print(f"❌ Camera {target_cam} not found") 
            continue
        
        # Get common timesteps between the two cameras
        source_timesteps = set(camera_frames[source_cam].keys())
        target_timesteps = set(camera_frames[target_cam].keys())
        common_timesteps = sorted(source_timesteps.intersection(target_timesteps))
        
        print(f"Common timesteps: {len(common_timesteps)}")
        if len(common_timesteps) < sequence_length:
            print(f"❌ Not enough common timesteps ({len(common_timesteps)} < {sequence_length})")
            continue
        
        # Find contiguous sequences
        sequences = []
        current_seq = [common_timesteps[0]]
        
        for i in range(1, len(common_timesteps)):
            if common_timesteps[i] == common_timesteps[i-1] + 1:
                # Contiguous
                current_seq.append(common_timesteps[i])
            else:
                # Gap found, save current sequence if long enough
                if len(current_seq) >= sequence_length:
                    sequences.append(current_seq.copy())
                current_seq = [common_timesteps[i]]
        
        # Don't forget the last sequence
        if len(current_seq) >= sequence_length:
            sequences.append(current_seq)
        
        print(f"Found {len(sequences)} contiguous sequences of length >= {sequence_length}")
        
        # Extract sequences of exact length
        valid_sequences = []
        for seq in sequences:
            # Extract all possible subsequences of desired length
            for start_idx in range(len(seq) - sequence_length + 1):
                subseq = seq[start_idx:start_idx + sequence_length]
                
                # Build frame info for this subsequence
                sequence_info = {
                    'timesteps': subseq,
                    'source_frames': [],
                    'target_frames': []
                }
                
                for timestep in subseq:
                    # Source camera frame
                    source_info = camera_frames[source_cam][timestep]
                    sequence_info['source_frames'].append({
                        'timestep': timestep,
                        'frame_name': source_info['frame_name'],
                        'dataset': source_info['dataset'],
                        'dataset_idx': source_info['dataset_idx']
                    })
                    
                    # Target camera frame
                    target_info = camera_frames[target_cam][timestep]
                    sequence_info['target_frames'].append({
                        'timestep': timestep,
                        'frame_name': target_info['frame_name'],
                        'dataset': target_info['dataset'],
                        'dataset_idx': target_info['dataset_idx']
                    })
                
                valid_sequences.append(sequence_info)
        
        results[(source_cam, target_cam)] = valid_sequences
        print(f"✓ Found {len(valid_sequences)} valid sequences of length {sequence_length}")
        
        # Show first sequence as example
        if valid_sequences:
            first_seq = valid_sequences[0]
            print(f"First sequence example:")
            print(f"  Timesteps: {first_seq['timesteps'][0]} to {first_seq['timesteps'][-1]}")
            print(f"  Source frames: {first_seq['source_frames'][0]['frame_name']} to {first_seq['source_frames'][-1]['frame_name']}")
            print(f"  Target frames: {first_seq['target_frames'][0]['frame_name']} to {first_seq['target_frames'][-1]['frame_name']}")
    
    return results

# Usage
sequences = find_contiguous_sequences(train_dataset, test_dataset, camera_pairs=[(0, 1), (0, 2)], sequence_length=49)

# Display summary
print(f"\n{'='*50}")
print("SUMMARY")
print(f"{'='*50}")

for camera_pair, seqs in sequences.items():
    source_cam, target_cam = camera_pair
    print(f"Camera {source_cam} -> Camera {target_cam}: {len(seqs)} sequences")

# Select and display a specific sequence
def select_sequence(sequences, camera_pair=(0, 1), sequence_idx=0):
    """Select a specific sequence and display details"""
    
    if camera_pair not in sequences:
        print(f"Camera pair {camera_pair} not found")
        return None
        
    if sequence_idx >= len(sequences[camera_pair]):
        print(f"Sequence index {sequence_idx} out of range (max: {len(sequences[camera_pair])-1})")
        return None
    
    selected = sequences[camera_pair][sequence_idx]
    
    print(f"\n=== Selected Sequence: Camera {camera_pair[0]} -> Camera {camera_pair[1]}, Index {sequence_idx} ===")
    print(f"Timesteps: {selected['timesteps'][0]} to {selected['timesteps'][-1]}")
    print(f"Length: {len(selected['timesteps'])}")
    
    print("\nSource camera frames (first 5):")
    for i, frame_info in enumerate(selected['source_frames'][:5]):
        print(f"  {i:2d}: {frame_info['frame_name']} (dataset: {frame_info['dataset']}, idx: {frame_info['dataset_idx']})")
    
    print("\nTarget camera frames (first 5):")
    for i, frame_info in enumerate(selected['target_frames'][:5]):
        print(f"  {i:2d}: {frame_info['frame_name']} (dataset: {frame_info['dataset']}, idx: {frame_info['dataset_idx']})")
    
    return selected

# Example: Select first sequence for camera 0->1
if (0, 1) in sequences and sequences[(0, 1)]:
    selected_sequence = select_sequence(sequences, camera_pair=(0, 1), sequence_idx=0)

# Example: Select first sequence for camera 0->2  
if (0, 2) in sequences and sequences[(0, 2)]:
    selected_sequence = select_sequence(sequences, camera_pair=(0, 2), sequence_idx=0)

Available cameras: [0, 1, 2]
Camera 0: 277 frames, timesteps 0-276
Camera 1: 219 frames, timesteps 0-276
Camera 2: 200 frames, timesteps 77-276

=== Checking camera pair (0, 1) ===
Common timesteps: 219
Found 2 contiguous sequences of length >= 49
✓ Found 123 valid sequences of length 49
First sequence example:
  Timesteps: 0 to 48
  Source frames: 0_00000 to 0_00048
  Target frames: 1_00000 to 1_00048

=== Checking camera pair (0, 2) ===
Common timesteps: 200
Found 1 contiguous sequences of length >= 49
✓ Found 152 valid sequences of length 49
First sequence example:
  Timesteps: 77 to 125
  Source frames: 0_00077 to 0_00125
  Target frames: 2_00077 to 2_00125

SUMMARY
Camera 0 -> Camera 1: 123 sequences
Camera 0 -> Camera 2: 152 sequences

=== Selected Sequence: Camera 0 -> Camera 1, Index 0 ===
Timesteps: 0 to 48
Length: 49

Source camera frames (first 5):
   0: 0_00000 (dataset: train, idx: 0)
   1: 0_00001 (dataset: train, idx: 1)
   2: 0_00002 (dataset: train, idx: 2)
   3: 0_000

In [9]:
sequences[(0, 1)][0]

{'timesteps': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48],
 'source_frames': [{'timestep': 0,
   'frame_name': '0_00000',
   'dataset': 'train',
   'dataset_idx': 0},
  {'timestep': 1,
   'frame_name': '0_00001',
   'dataset': 'train',
   'dataset_idx': 1},
  {'timestep': 2,
   'frame_name': '0_00002',
   'dataset': 'train',
   'dataset_idx': 2},
  {'timestep': 3,
   'frame_name': '0_00003',
   'dataset': 'train',
   'dataset_idx': 3},
  {'timestep': 4,
   'frame_name': '0_00004',
   'dataset': 'train',
   'dataset_idx': 4},
  {'timestep': 5,
   'frame_name': '0_00005',
   'dataset': 'train',
   'dataset_idx': 5},
  {'timestep': 6,
   'frame_name': '0_00006',
   'dataset': 'train',
   'dataset_idx': 6},
  {'timestep': 7,
   'frame_name': '0_00007',
  